# Importación de librerías

In [1]:
import numpy as np
import pandas as pd
import os
import csv
import random
from PIL import Image
from skimage.transform import resize

from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras import layers

import matplotlib.pyplot as plt

# Carga de Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Lectura de los datos

In [3]:
# Carga de los dataframes de train, test y validation
df_train = pd.read_csv("/content/drive/MyDrive/TFM/train_labels.csv")
df_test = pd.read_csv("/content/drive/MyDrive/TFM/test_labels.csv")
df_validation = pd.read_csv("/content/drive/MyDrive/TFM/validation_labels.csv")

print("El número de registros en df_train es: " + str(len(df_train)))
print("El número de registros en df_test es: " + str(len(df_test)))
print("El número de registros en df_validation es: " + str(len(df_validation)))

El número de registros en df_train es: 32144
El número de registros en df_test es: 3000
El número de registros en df_validation es: 3000


In [4]:
df_train.head()

,PatientId,Target
0,f2e8cacc-b217-4e82-93ba-33bf5e04e94c,1
1,4b46098e-ea85-41de-9534-0a645ec9b121,0
2,dee054ff-0e1a-4167-b814-cbf339cf689c,0
3,d798aa5b-28ef-463e-81a1-c05c83dcdebb,0
4,2e13eff6-7bb4-4b45-846f-1522888dbfe1,1


In [5]:
df_test.head()

,PatientId,Target
0,49f0fa25-89cd-41a7-9c70-4d7d8d9be572,1
1,eb10b7c7-fc94-488a-a5a9-704876d78ebb,1
2,c56b1137-e4d4-4139-94ba-b287ca43e318,0
3,17f68ac3-2385-4b13-b8c0-66ba8b343205,1
4,0c89ca38-27ca-4cd0-88cb-18093fdcb04b,1


In [6]:
df_validation.head()

,PatientId,Target
0,b35d57ee-f22b-4c8c-b959-3ee8eecef555,1
1,09ea637f-0f6a-427e-8dad-858330a2498e,0
2,0cbc601f-91f0-4f86-b780-ffeac24471c7,1
3,47d188c2-6a54-4548-975e-85a63643b08b,0
4,172f8b71-1d19-4a36-ae9f-41ae528c72be,1


## Comprobación de los datos

Vamos a comprobar que las imágenes que se encuentran en cada directorio, es decir, en train, validation, y test, también están en los dataframes correspondientes.

Se hace esta comprobación básicamente para saber si se han subido de forma correcta las imágenes.

In [ ]:
def check_images(dir, df):
  files = os.listdir(dir)
  num_elements = len(df)
  num_images = len(files)

  # Recorremos el dataframe correspondiente y comprobamos si están las imágenes
  for patientId in df["PatientId"]:
    if not patientId + ".png" in files:
      return False

  # Comprobamoos que hay el mismo número de registros que de imágenes
  if num_elements != num_images:
    return False

  return True

In [ ]:
print("Train: " + str(check_images("/content/drive/MyDrive/TFM/train_images", df_train)))
print("Test: " + str(check_images("/content/drive/MyDrive/TFM/test_images", df_test)))
print("Validation: " + str(check_images("/content/drive/MyDrive/TFM/validation_images", df_validation)))

Train: True
Test: True
Validation: True


# Data Generator

En este punto vamos a resolver la problemática de la carga de los datos en memoria. El dataset de entrenamiento presenta 32144 imágenes con resolución 1024x1024, lo cual hace que haya un elevado consumo de memoria, hasta tal punto que puede que Google Colab no sea capaz de soportar.

Para solventar este problema creamos un data generator, el cual se encarga de cargar en memoria pequeños grupos de imágenes según se vayan utilizando, es decir, dependiendo del tamaño del batch.

La salida que proporciona el data generator es un cojunto de imágenes junto con la variable objetivo.


In [42]:
# https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

class DataGenerator(keras.utils.Sequence):

  # Constructor
  def __init__(self, folder, batch_size=32, image_size=256, shuffle=True, predict=False):
    self.folder = folder
    self.filenames = os.listdir(folder)
    self.batch_size = batch_size
    self.image_size = image_size
    self.shuffle = shuffle
    self.predict = predict
    self.on_epoch_end()
    

  # Carga y Transformación de la imágenes para training
  # filename: es el nombre del archivo de la imagen, es decir, con png
  def __train__(self, filename):
    # Cargamos la imagen original
    img = np.array(Image.open(os.path.join(self.folder, filename)))

    # Cargamos la variable objetivo
    target = df_train[df_train["PatientId"] == filename.split(".")[0]]["Target"].item()

    # Reducción de la escala de la imagen
    img = resize(img, (self.image_size, self.image_size), mode="reflect")

    # Normalizamos
    # img_min = img.min()
    # img_max = img.max()
    # img_norm = (img - img_min) / (img_max - img_min)

    # Expandimos las dimensiones (self.image_size, self.image_size, 1)
    # img_norm = np.expand_dims(img_norm, -1)
    img = np.expand_dims(img, -1)

    return img, target


  # Carga y transformación de las imágenes para testing
  def __test__(self, filename):
    # Cargamos la imagen original
    img = np.array(Image.open(os.path.join(self.folder, filename)))

    # Reducción de la escala de la imagen
    img = resize(img, (self.image_size, self.image_size), mode="reflect")

    # Normalizamos
    # img_min = img.min()
    # img_max = img.max()
    # img_norm = (img - img_min) / (img_max - img_min)

    # Expandimos las dimensiones (self.image_size, self.image_size, 1)
    # img_norm = np.expand_dims(img_norm, -1)
    img = np.expand_dims(img, -1)

    return img

  
  # Método encargado de generar el batch
  def __getitem__(self, index):
    # Generación de los nombres de archivos pertenecientes al batch
    filenames_batch = self.filenames[index*self.batch_size:(index+1)*self.batch_size]

    if self.predict:
      # Modo testing
      imgs = [self.__test__(filename) for filename in filenames_batch]
      imgs = np.array(imgs)
      return imgs, filenames_batch

    else:
      # Modo training
      items = [self.__train__(filename) for filename in filenames_batch]
      imgs, targets = zip(*items)
      imgs = np.array(imgs)
      targets = np.expand_dims(np.array(targets), -1)
      return imgs, targets

  # Método encargado de mezclar nos nombres de archivos, para así dotar de una mayor aleatoriedad
  def on_epoch_end(self):
    if self.shuffle:
      random.shuffle(self.filenames)

  # Método para controlar el tamaño del 
  def __len__(self):
    return int(len(self.filenames) / self.batch_size)

In [48]:
dg = DataGenerator(folder=TRAIN_FOLDER, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
dg.__len__()

1004

In [79]:
filenames_batch = dg.filenames[0*dg.batch_size:(0+1)*dg.batch_size]
dic = {}

for filename in filenames_batch:
  dic[filename.split(".")[0]] = np.array(Image.open(os.path.join(dg.folder, filename)))

In [85]:
import time

inicio_1 = 0
inicio_2 = 0
inicio_3 = 0
inicio_4 = 0
inicio_5 = 0
inicio_6 = 0
fin_1 = 0
fin_2 = 0
fin_3 = 0
fin_4 = 0
fin_5 = 0
fin_6 = 0



inicio_1 = time.time()

filenames_batch = dg.filenames[0*dg.batch_size:(0+1)*dg.batch_size]
items = [] #[dg.__train__(filename) for filename in filenames_batch]
for filename in filenames_batch:

  inicio_2 += time.time()
  # Cargamos la imagen original
  # img = np.array(Image.open(os.path.join(dg.folder, filename)))
  img = dic[filename.split(".")[0]]
  fin_2 += time.time()

  inicio_3 += time.time()
  # Cargamos la variable objetivo
  target = df_train[df_train["PatientId"] == filename.split(".")[0]]["Target"].item()
  fin_3 += time.time()

  # Reducción de la escala de la imagen
  inicio_4 += time.time()
  img = resize(img, (dg.image_size, dg.image_size), mode="reflect")
  fin_4 += time.time()

  # Normalizamos
  # img_min = img.min()
  # img_max = img.max()
  # img_norm = (img - img_min) / (img_max - img_min)

  # Expandimos las dimensiones (self.image_size, self.image_size, 1)
  # img_norm = np.expand_dims(img_norm, -1)
  inicio_5 = time.time()
  img = np.expand_dims(img, -1)
  fin_5 = time.time()

  items.append([img, target])

fin_1 = time.time()



inicio_6 = time.time()

imgs, targets = zip(*items)
imgs = np.array(imgs)
targets = np.expand_dims(np.array(targets), -1)

fin_6 = time.time()

tramo_1 = fin_1-inicio_1
tramo_2 = fin_2-inicio_2
tramo_3 = fin_3-inicio_3
tramo_4 = fin_4-inicio_4
tramo_5 = fin_5-inicio_5
tramo_6 = fin_6-inicio_6

print("*"*50)
print("El tramo 1 dura (s): " + str(tramo_1))
print("El tramo 2 dura (s): " + str(tramo_2))
print("El tramo 3 dura (s): " + str(tramo_3))
print("El tramo 4 dura (s): " + str(tramo_4))
print("El tramo 5 dura (s): " + str(tramo_5))
print("  => Suma de subtramos (s):" +  str(tramo_2 + tramo_3 + tramo_4 + tramo_5))
print("*"*50)
print("El tramo 6 dura (s): " + str(tramo_6))


**************************************************
El tramo 1 dura (s): 1.671962022781372
El tramo 2 dura (s): 0.000335693359375
El tramo 3 dura (s): 0.16327667236328125
El tramo 4 dura (s): 1.5058517456054688
El tramo 5 dura (s): 5.412101745605469e-05
  => Suma de subtramos (s):1.669518232345581
**************************************************
El tramo 6 dura (s): 0.0027468204498291016


In [47]:
for i in range(dg.__len__()):
  print(i)
  dg.__getitem__(i)

0
1


KeyboardInterrupt: ignored

In [15]:
filenames = os.listdir(TRAIN_FOLDER)

len(filenames)

32144

In [33]:
aux = []
for filename in filenames:
  aux.append(df_train[df_train["PatientId"] == filename.split(".")[0]]["Target"].item())
print(len(aux))

32144


# Red Neuronal Convolucional

El siguiente paso es crear la red neuronal convolucional, para ello vamos a hacer uso de `EfficientNet`, en nuestro caso usaremos `EfficientNetB0`.

Al usar `EfficientNetB0` estamos "limitados" a que la resolución de las imágenes sea de 224x224, en vez de 1024x1024 que era el tamaño original.

Otro punto a destacar es que la imagen que introduzcamos al modelo no hace falta normalizarla, ya que se encarga `EfficientNetB0` de hacerlo.

## Hiperparámetros

In [8]:
# Data generator
IMG_SIZE = 224
BATCH_SIZE = 32

# CNN
NUM_CLASSES = 2
LEARNING_RATE = 1e-3
EPOCHS = 20

# EXTRA
TRAIN_FOLDER = "/content/drive/MyDrive/TFM/train_images"
TEST_FOLDER = "/content/drive/MyDrive/TFM/test_images"
VALID_FOLDER = "/content/drive/MyDrive/TFM/validation_images"

## Data augmentation

El siguiente punto es definir el data augmentation, para así mejorar la variabilidad del entrenamiento y por lo tanto la precisión de la red neuronal.

In [9]:
img_augmentation = Sequential(
    [
     layers.RandomRotation(factor=0.05),
     layers.RandomTranslation(height_factor=0.05, width_factor=0.02),
     layers.RandomFlip("horizontal"),
     layers.RandomContrast(factor=0.05),
    ],
    name = "img_augmentation"
)

## CNN y transfer learning

El siguiente paso es crear la red neuronal convolucional y aplicar transefer learning.

El transfer learning lo que significa es que vamos a tener dos modelos, un modelo base con pesos ya pre-entrenados con otro tipo de problemas, y un modelo nuevo basado en el modelo base.

De forma resumida lo que se va a hacer es lo siguiente:
* 1º: vamos a inicializar el modelo base con pesos ya pre-entrenados.
* 2º: se van a congelar todos los layers del modelo base con `trainable = False`.
* 3º: Creamos un nuevo modelo después de la salida del modelo base.
* 4º: Entrenamos el nuevo modeloo con el dataset.

In [10]:
# Método que se encarga de generar el modelo
# https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/#transfer-learning-from-pretrained-weights
# https://stackoverflow.com/questions/51995977/how-can-i-use-a-pre-trained-neural-network-with-grayscale-images
def build_model():
  inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 1))
  x = img_augmentation(inputs)
  x = layers.Concatenate()([x, x, x])  
  model = EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")

  # Freeze the pretrained weights
  model.trainable = False

  # Rebuild top
  x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
  x = layers.BatchNormalization()(x)

  top_dropout_rate = 0.2
  x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
  # outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)
  outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)

  # Compile
  model = keras.Model(inputs, outputs, name="EfficientNet")
  optimizer = Adam(learning_rate=LEARNING_RATE)
  model.compile(
      optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
  )
  return model

## Training

In [12]:
# Obtenemos los generadores para training y validaton
train_gen = DataGenerator(folder=TRAIN_FOLDER, batch_size=BATCH_SIZE, image_size=IMG_SIZE, shuffle=True, predict=False)
valid_gen = DataGenerator(folder=VALID_FOLDER, batch_size=BATCH_SIZE, image_size=IMG_SIZE, shuffle=False, predict=False)

In [13]:
# Construimos el modelo (tanto el modelo base como el nuestro)
model = build_model()

16719872/16705208 [==============================] - 0s 0us/step


In [14]:
# Entrenamos la red
history = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=EPOCHS,
    workers=4
)

Epoch 1/20
1004/1004 [==============================] - ETA: 0s - loss: 0.7280 - accuracy: 0.5030

ValueError: ignored